參考： https://github.com/OmniXRI/Colab_OpenCV_Display/blob/main/Colab_OpenCV_Display.ipynb

In [ ]:
#安裝 translate 套件
!pip install translate

In [ ]:
import datetime
import matplotlib.pyplot as plt # 導入matplotlib.pyplot函式庫
import textwrap
import unicodedata
import requests
import configparser
from PIL import ImageFont, ImageDraw, Image    # 載入 PIL 相關函式庫
from typing import List, Tuple
from lxml import etree
import re

elder_config.ini ，內容如下

[unsplash]
client_id = 
[openai]
api_key = 
[cwb]
authorization = 
[ninjas]
api_key = 

In [ ]:
def get_formatted_date():
    today = datetime.date.today()

    week_dict = {0: '星期一', 1: '星期二', 2: '星期三', 3: '星期四', 4: '星期五', 5: '星期六', 6: '星期日'}
    weekday = week_dict[today.weekday()]
    formatted_date = f"{today.strftime('%Y.%m.%d')} {weekday}"
    
    return formatted_date

In [ ]:
#當字串中有中文時，每個中文字視為2個英文字母
def get_max_count_perline(text: str):
    max_count = 0
    lines = text.split('\n')
    for line in lines:
        count = 0
        for char in line:
            if unicodedata.category(char) == 'Lo': 
                count += 0.5
            else:
                count += 1
        if count > max_count:
          max_count = count
    return round(max_count)

In [ ]:
#TOFIX: TextWrapper 的斷行判斷不夠聰明
def wrap_text_with_newline(text: str, wrap_width: int):
    # 先將 \n 替換為空白字元
    text = text.replace('\n', ' ')
    # 將斷行寬度調整為 wrap_width - 1，因為要留一個位置給 \n 字元
    wrapper = textwrap.TextWrapper(width=wrap_width - 1)
    lines = wrapper.wrap(text)
    # 將每行的末尾加上 \n 字元
    lines_with_newline = [line + '\n' for line in lines]

    return lines_with_newline

In [ ]:
# 計算多行文字寬度及高度
def get_total_size(font: ImageFont, lines: List[str], width_per_char: int) -> Tuple[int, int]:
  max_width = 0
  total_height = 0
  for line in lines:
      width = len(line) * width_per_char
      max_width = max(max_width, width)
      height = font.getsize(line)[1]
      total_height += int(round(height * 1.4))
  return max_width, total_height

In [ ]:
def generate_title_image(title: str, font_file: str, font_size: int, stroke_width: int, stroke_color: tuple, fill_color: tuple):
    # 設定字型
    font = ImageFont.truetype(font_file, font_size)

    # (單行)文字寬度和高度
    width, height = font.getsize(title)

    img_title = Image.new('RGBA', (width+10, height+10), color=(255, 255, 255, 0))
    draw_title = ImageDraw.Draw(img_title)
    # 繪製標題
    draw_title.text((5, 5), title, font=font, fill=fill_color, stroke_width=stroke_width, stroke_fill=stroke_color)
    
    # 顯示和儲存圖片
    plt.clf()
    plt.axis("off") # 設定關閉XY軸刻尺
    plt.imshow(img_title) # 繪製單張影像
    # 將圖表存檔時使用緊湊的邊界框
    plt.savefig('title.png', bbox_inches='tight', pad_inches=0.5)
    plt.show() # 顯示影像

    return img_title

In [ ]:
# 名言佳句
def generate_image_with_quotes(text: str, font_file: str, font_size: int, stroke_width: int, stroke_color: tuple, fill_color: tuple, wrap_width: int):
    # 設定字型
    font = ImageFont.truetype(font_file, font_size)
    lines = wrap_text_with_newline(text, wrap_width)

    width_per_char = font.getsize('安')[0]
    max_width, total_height = get_total_size(font, lines, width_per_char)

    img_quotes = Image.new('RGBA', (max_width, total_height), color=(255, 255, 255, 0))
    draw_quotes = ImageDraw.Draw(img_quotes)

    # 繪製文字
    y = 0
    for line in lines:
        width, height = font.getsize(line)
        draw_quotes.text((0, y), line, font=font, fill=fill_color, stroke_width=stroke_width, stroke_fill=stroke_color)
        y += height + int(round(height * 0.4))

    plt.axis("off") # 設定關閉XY軸刻尺
    plt.imshow(img_quotes) # 繪製單張影像

    # 將圖表存檔時使用緊湊的邊界框
    plt.savefig('quotes.png', bbox_inches='tight', pad_inches=0.8)

    plt.show() # 顯示影像
    return img_quotes

In [ ]:
# 讀取外部配置檔案
config = configparser.ConfigParser()
config.read('elder_config.ini')

In [ ]:
title_config = {
    'title': "早安",
    'font_file': 'Iansui0.92-Regular.ttf',
    'font_size': 96,
    'stroke_width': 5,
    'stroke_color': (255, 255, 255, 255),
    'fill_color': (64, 128, 0, 255)
}
img_title = generate_title_image(**title_config)

In [ ]:
text = "祝您今天有個美好的開始。\nGood morning and have a nice day!\n您的摯友~\n" + str(get_formatted_date())

quotes_config = {
    'text': text,
    'font_file': 'Iansui0.92-Regular.ttf',
    'font_size': 48,
    'stroke_width': 5,
    'stroke_color': (255, 255, 255, 255),
    'fill_color': (0, 0, 0, 255),
    'wrap_width': get_max_count_perline(text)
}

img_quotes = generate_image_with_quotes(**quotes_config)

In [ ]:
# 讀取底圖
img_bg = Image.open("animal.webp").convert('RGBA')
img_bg.paste(img_title, (30, 96), mask=img_title)
img_bg.paste(img_quotes, (30, img_bg.height - img_quotes.height - 30), mask=img_quotes)

# 儲存合成後的圖片
img_bg.save("早安.png")

In [98]:
# 使用 unsplash 圖庫
url = 'https://api.unsplash.com/photos/random'
params = {
    'query': 'buddha',
    'client_id': config.get('unsplash', 'client_id')
}

response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
    image_url = data['urls']['regular']
    image_data = requests.get(image_url).content

    with open('unsplash.jpeg', 'wb') as f:
        f.write(image_data)
else:
    print('Failed to get image:', response.status_code)

# 讀取底圖
img_bg = Image.open("unsplash.jpeg").convert('RGBA')

img_bg.paste(img_title, (30, 96), mask=img_title)
img_bg.paste(img_quotes, (30, img_bg.height - img_quotes.height - 30), mask=img_quotes)

# 儲存合成後的圖片
img_bg.save("unsplash_早安.png")

In [ ]:
#DALL-E ver.1
import requests
import json
from PIL import Image

api_endpoint = "https://api.openai.com/v1/images/generations"
api_key = config.get('openai', 'api_key')

# 設定請求標頭
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

# 設定模型 ID 和輸入文本
model = "image-alpha-001"
prompt = "Golden buddha statue with cherry blossom. compact baby face, Mediation and zen concept, high quality in Chinese style, full body, dynamic pose, perfect anatomy, centered, freedom, soul, approach to perfection, cell shading, 4k , cinematic dramatic atmosphere, global illumination, detailed and intricate environment, fluid and sharp focus"

# 設定請求內容
data = {
    "model": model,
    "prompt": prompt,
    "num_images": 1,
    "size": "1024x1024",
    "response_format": "url"
}

# 發送請求
response = requests.post(api_endpoint, headers=headers, data=json.dumps(data))

# 解析回傳結果
if response.status_code == 200:
    result = response.json()
    image_url = result['data'][0]['url']
    print(f"Generated image URL: {image_url}")
else:
    print(f"Failed to generate image: {response.status_code} - {response.text}")

# 提取文本生成的圖片 URL
print(response)
# 下載並儲存結果圖片
dalle = requests.get(image_url)
with open("dalle.jpeg", "wb") as f:
    f.write(dalle.content)

# 讀取底圖
img_bg = Image.open("dalle.jpeg").convert('RGBA')
img_bg.paste(img_title, (30, 96), mask=img_title)
img_bg.paste(img_quotes, (30, img_bg.height - img_quotes.height - 30), mask=img_quotes)

# 儲存合成後的圖片
img_bg.save("dalle_早安.png")


In [ ]:
#DALL-E 2

In [ ]:
# 中央氣象局之當地縣縣市氣象
def get_local_weather(location):
    """
    :param location: 縣市名稱，例如 "臺北市"、"新北市"、"高雄市"
    :return: 當地天氣資訊的 JSON 格式資料
    """
    # API 連結
    url = 'https://opendata.cwb.gov.tw/api/v1/rest/datastore/F-C0032-001'
    # 臺灣各鄉鎮市區預報資料-臺灣各鄉鎮市區未來2天(逐3小時)及未來1週天氣預報
    # https://opendata.cwb.gov.tw/api/v1/rest/datastore/F-D0047-033

    # 請在下方填入您的授權碼
    authorization = config.get('cwb', 'authorization')

    # 參數設定
    params = {
        'locationName': location,
        'Authorization': authorization,
        'format': 'JSON'
    }

    try:
        # 發送 GET 請求
        response = requests.get(url, params=params)
        response.raise_for_status()  # 檢查請求是否成功

        # 取得 JSON 格式資料
        weather_data = response.json()

        return weather_data

    except requests.exceptions.RequestException as e:
        print('發生錯誤: ', e)
        return None

In [ ]:
# 測試範例：取得該縣市當地天氣資訊
location = '屏東縣'
weather_data = get_local_weather(location)

# 取得屏東縣的氣象資料
location = weather_data['records']['location'][0]
location_name = location['locationName']
weather_elements = location['weatherElement']

# 取得氣候狀態
wx = None
# 取得溫度資料
min_t = None
max_t = None
for element in weather_elements:
    if element['elementName'] == 'Wx':
        wx = element['time'][0]['parameter']['parameterName']
    elif element['elementName'] == 'MinT':
        min_t = element['time'][0]['parameter']['parameterName']
    elif element['elementName'] == 'MaxT':
        max_t = element['time'][0]['parameter']['parameterName']

# 顯示氣候資料
quote_text = f'地區名稱: {location_name}\n氣候: {wx}\n氣溫：{min_t}~{max_t}°C'

# 印出名言(以氣象資料回報)
quotes_config['text'] = quote_text
quotes_config['wrap_width'] = 12

img_quotes = generate_image_with_quotes(**quotes_config)

In [ ]:
# 有很多種的 API 服務：https://api-ninjas.com/
# 取得名言佳句
def get_ninjas_quote(category, api_key):
    api_url = f"https://api.api-ninjas.com/v1/quotes?category={category}"
    response = requests.get(api_url, headers={'X-Api-Key': api_key})
    if response.status_code == requests.codes.ok:
        data = response.json()[0]
        quote_text = data["quote"]
        author_name = data["author"]
        return quote_text, author_name
    else:
        print("Error:", response.status_code, response.text)
        return None, None

In [ ]:
# 設定要取得的名言佳句類別
category = 'morning'
api_key = config.get('ninjas', 'api_key')

# 呼叫函式取得名言佳句文字和作者
quote_text, author_name = get_ninjas_quote(category, api_key)

# 印出名言
if quote_text and author_name:
  quotes_config['text'] = f"{quote_text} -- {author_name}"
  quotes_config['wrap_width'] = 40
  print(f"{quote_text} -- {author_name}")

img_quotes = generate_image_with_quotes(**quotes_config)

In [ ]:
from translate import Translator

translator= Translator(to_lang="zh-hant")

quote_text = translator.translate(quote_text)

# 顯示翻譯結果
print("中文翻譯:", quote_text)

# 印出名言
if quote_text and author_name:
  quotes_config['text'] = f"{quote_text} -- {author_name}"
  quotes_config['wrap_width'] = 20
  print(f"{quote_text} -- {author_name}")

img_quotes = generate_image_with_quotes(**quotes_config)

In [ ]:
# 讀取底圖
img_bg = Image.open("unsplash.jpeg").convert('RGBA')

img_bg.paste(img_title, (30, 96), mask=img_title)
img_bg.paste(img_quotes, (30, img_bg.height - img_quotes.height - 30), mask=img_quotes)

# 儲存合成後的圖片
img_bg.save("unsplash_早安.png")

In [ ]:
# 請求網頁
def requestHtml(url, data=None):
  s = requests.Session()
  r = s.post(url, data=data)  # 使用 POST 方法
  r.encoding = r.apparent_encoding
  # print(r.text)
  return etree.HTML(r.text)

In [ ]:
# 孔明神數 https://pay.ncc.com.tw/s.php?word3=nccsoft&ID=ncc&p=ba_bm
# 設定查詢的 URL
url = 'https://pay.ncc.com.tw/s.php?word3=nccsoft&ID=ncc&p=ba_bm'
spell = '問吉凶'

# 設定 POST 方法的參數
data = {
    'ps' : '',
    'chgps': '0',
    'ShowAdd': '1',
    '_Name': spell
}

# 發送 POST 請求
dom = requestHtml(url, data)

poem = dom.xpath("//p[contains(text(),'籤詩')]/text()")[0].replace("籤詩: ", "")
print(poem)
# 使用正則表達式取代標點符號為空白
poem = re.sub(r'[^\w\s]', ' ', poem)

quotes_config['text'] = poem
quotes_config['wrap_width'] = 8

img_quotes = generate_image_with_quotes(**quotes_config)

# 讀取底圖
img_bg = Image.open("unsplash.jpeg").convert('RGBA')

img_bg.paste(img_title, (30, 96), mask=img_title)
img_bg.paste(img_quotes, (30, img_bg.height - img_quotes.height - 30), mask=img_quotes)

# 儲存合成後的圖片
img_bg.save("unsplash_吉凶.png")